In [2]:
import os
import pathlib

ROOT_PATH = pathlib.Path(__name__).resolve().parent.parent
print(ROOT_PATH)
RANDOM_SEED = 42

/Users/andrey/PycharmProjects/vector-search-hse


In [3]:
os.chdir(ROOT_PATH)
os.getcwd()

'/Users/andrey/PycharmProjects/vector-search-hse'

In [4]:
# Загрузка лейблов
import pandas as pd

df = pd.read_csv(ROOT_PATH / 'data' / 'raw_labels.tsv', sep='\t')
df = df.drop(columns=['Комментарий'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           322 non-null    object
 1   video_name   322 non-null    object
 2   text_labels  322 non-null    object
dtypes: object(3)
memory usage: 7.7+ KB


In [7]:
len(df.video_name.unique())

322

In [8]:
df = df[df['text_labels'].isna() == False]
df['tags'] = df['text_labels'].apply(lambda x: {w.strip() for w in x.split(',')})
tags_df = df[['video_name', 'tags']].explode('tags')
tags_df.columns =  ['video_name', 'tag_ru']

In [9]:
!pip install deep-translator


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ru', target='en')

def translate(text: str) -> str:
    translated = translator.translate(text)
    return translated



In [11]:
from tqdm import tqdm

tags_ru = set(tags_df['tag_ru'].values)

print('Total tags:', len(tags_ru))

tags_map = {}
for tag in tqdm(tags_ru):
    translated_tag = translate(tag)
    tags_map[tag] = translated_tag

Total tags: 299


100%|██████████| 299/299 [00:50<00:00,  5.96it/s]


In [16]:
tags_df['tag_en'] = tags_df['tag_ru'].map(tags_map)
tags_df.head()

,video_name,tag_ru,tag_en
0,IMG_0903.MOV,космос,space
0,IMG_0903.MOV,усы,mustache
0,IMG_0903.MOV,мужчина,man
0,IMG_0903.MOV,NASA,NASA
1,IMG_0718.MOV,актёр,actor


In [18]:
# Подсчет статистики по тегам
tags_df['tag_en'].value_counts(normalize=True, ascending=False)

tag_en
man         0.117124
woman       0.050889
glasses     0.042003
food        0.040388
mustache    0.034733
              ...   
cars        0.000808
alpaca      0.000808
cosplay     0.000808
ladle       0.000808
sandwich    0.000808
Name: proportion, Length: 290, dtype: float64

In [19]:
selected_tags = tags_df[tags_df['tag_en'].isna() == False]

In [20]:
# Сохранение тегов
selected_tags.to_parquet('data/tags.parquet', index=False)

In [18]:
test_queries = tags_df.groupby('tag_en')['video_name'].agg(list).reset_index()
test_queries['ttl_videos'] = test_queries['video_name'].apply(lambda x: len(x))

test_queries = test_queries[test_queries['ttl_videos'] > 10]
test_queries

,tag_en,video_name,ttl_videos
17,People,"[IMG_1002.MOV, IMG_0719.MOV, IMG_0928.MOV, IMG...",23
30,animal,"[IMG_1823.MOV, IMG_1016.MOV, IMG_0731.MOV, IMG...",35
48,beard,"[IMG_1177.MOV, IMG_0928.MOV, IMG_1834.MOV, IMG...",30
67,car,"[IMG_1564.MOV, IMG_1835.MOV, IMG_0726.MOV, IMG...",22
69,cartoon,"[IMG_1767.MOV, IMG_0874.MOV, IMG_1564.MOV, IMG...",23
70,cat,"[IMG_0733.MOV, IMG_0732.MOV, IMG_1563.MOV, IMG...",17
85,computer game,"[IMG_1404.MOV, IMG_1100.MOV, IMG_0752.MOV, IMG...",12
88,cooking,"[IMG_1834.MOV, IMG_1767.MOV, IMG_1561.MOV, IMG...",36
102,dog,"[IMG_0731.MOV, IMG_0733.MOV, IMG_1558.MOV, IMG...",19
105,drawing,"[IMG_0737.MOV, IMG_1365.MOV, IMG_0815.MOV, IMG...",21
